# Aula 28 – Detecção de Objetos

## 1. Introdução

Na aula passada vimos o algoritmo de Viola-Jones para detecção de objetos, mas aplicado à detecção de faces, pois foi sua origem e, dada sua performance, é onde também tem maior utilidade. O algoritmo basicamente procura padrões de pixels que normalmente são identificados em faces, aplicando filtros retangulares em vários pontos da imagem procurando locais onde eles se “encaixam”. Tal algoritmo pode ser generalizado com filtros para objetos quaisquer e, para cada tipo de objeto que desejássemos detectar, teríamos um conjunto de filtros diferentes treinado.

A generalização do algoritmo, portanto, é relativamente pesada e depende fortemente dos filtros realmente representarem organizações típicas de pixels nos objetos a serem detectados, o que varia muito também com o modo como os objetos aparecem nas fotos. Por exemplo, quando detectamos faces, estamos tratando na maioria das vezes de imagens frontais, o que garante a existência dos padrões dos filtros. Quando o ângulo muda muito, o algoritmo começa a apresentar mais dificuldades, a não ser que também tenha sido treinado em outros casos, mas isso provavelmente aumentaria o número de filtros e, consequentemente, o peso computacional no processo de detecção.

O objetivo desta aula é apresentar um modelo mais genérico para detecção de objetos. É computacionalmente mais pesado, porém mais fácil de generalizar. Até a aula 26 aprendemos como poderíamos classificar imagens. Para tanto, era preciso assumir que toda a imagem analisada trazia apenas um objeto que queríamos classificar. Nesse caso, o processo tratava de dizer se a imagem era ou não de determinada classe. Agora queremos saber se uma imagem tem ou não o objeto, e não se ela é o objeto. A ideia é simples: recortar a imagem e, em cada pedaço, fazer a classificação. Se algum pedaço for positivo para a classe procurada, o algoritmo retorna que o objeto foi detectado e ainda é capaz de informar sua localização.

O problema passa a ser então como recortar a imagem, já que o objeto procurado pode estar em qualquer lugar. Existem várias ideias, mas todas partem de um princípio básico: uma janela que percorre a imagem (janela deslizante). Teoricamente, tal janela deveria percorrer todos os pixels e ter todos os tamanhos possíveis na imagem. Entretanto, isso seria extremamente custoso computacionalmente, inviabilizando o processo de detecção. Os diferentes algoritmos existentes, portanto, basicamente se concentram em otimizar os tamanhos e localizações das janelas (assumindo que a classificação já está resolvida). Para isso, os algoritmos trabalham em cima das seguintes observações:
    • Janelas com diferença de localização de 1 (ou poucos) pixel(s) recortam praticamente o mesmo canto da imagem, ou seja, não é razoável pensar em classificações diferentes para elas. Logo, na prática, podemos fazer deslocamentos maiores. O tamanho do deslocamento pode estar relacionado ao tamanho da janela ou ao tamanho típico do objeto procurado;
    • Janelas muito pequenas vão recortar pedaços da imagem que, se o objeto de fato estiver em tal localização, será tão pequeno que não será detectável (às vezes sequer por humanos). Ou seja, na prática, podemos definir tamanhos mínimos que sejam razoáveis para a detecção do objeto. Novamente, a decisão do tamanho costuma estar relacionada ao tamanho típico do objeto e o tamanho total da imagem.
    • Quando tratamos de fotos, principalmente, espera-se exista um determinado enquadramento para objetos de interesse, ou seja, também existem áreas típicas na imagem onde normalmente encontraremos os objetos que desejamos encontrar. Isso também limita as localizações das janelas úteis.
Tais observações ajudam na construção de algoritmos de “propostas de regiões”, que funcionam de forma estatística e também precisam ser treinados, pois a proposta depende muito da análise de como o objeto procurado normalmente se apresenta nas imagens. Alguns algoritmos também usam janelas fixas, isto é, sempre utilizam, independente do objeto, o mesmo conjunto de recortes para encontrar objetos, afinando o resultado após classificação, pois nenhuma janela pode representar a localização ideal do objeto.

Outro processo feito é a inferência de qual janela melhor captura o objeto detectado. Em alguns casos, mesmo após toda a limpeza para trabalhar apenas com os recortes úteis da imagem, um mesmo objeto aparece em mais de uma janela diferente. Sendo assim, o algoritmo também precisa decidir qual janela melhor “recorta” o objeto ou até mesmo definir um novo recorte “ideal” baseado nas janelas que deram positivo.

O processo de detecção de objetos pode envolver ainda técnicas mais precisas que vão além da classificação com localização, como:
    • Segmentação semântica: indica quais pixels na imagem pertencem ao objeto. Ou seja, o retorno não é simplesmente uma região retangular onde o objeto se encontra;
    • Segmentação de instância: quando mais de um objeto do mesmo tipo é detectado, o algoritmo é capaz de informar que são objetos diferentes de uma mesma classe.

Atualmente os melhores algoritmos para detecção de objetos utilizam Redes Neurais Convolucionais. E tais redes são utilizadas não apenas na fase da classificação, mas também para fazer a proposta de regiões. Na próxima aula veremos algumas arquiteturas de RNC's para detecção de objetos.

## 2. Material Complementar

http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture11.pdf

https://www.youtube.com/watch?v=nDPWywWRIRo

## 3. Exercícios

1. Pesquise e descreva como é feito o recorte da imagem para detecção de objetos nos seguintes métodos: YOLO (You Only Look Once), SSD (Single Shot Detector) e Faster R-CNN.



## SSD (Single shot detector)
SSD, discretiza o espaço de bounding boxes de saída em um conjunto padronizado de bboxes de diferentes taxas de aspecto (aspect ratios) e os escala de acordo com a localização do mapa de características identificado. Durante a predição, a rede gera scores para a presença de cada categoria de objeto em cada bbox padrão e produz ajustes para o bbox que nelhor se ajustar ao formato do objeto. Para isso, a rede combina predições geradas por múltiplos mapas de características com diferentes resoluções para naturalmente lidar com objetos de diferentes tamanhos. É um modelo simples se comparado a métodos que requerem a geração de propostas de objetos pois elimina completamente a geração de propostas e a subseqüente reamostragem de pixels ou características e encapsula todos os cálculos em uma única rede. É fácil de treinar por causa de sua simplicidade, sendo fácil de integrar em sistemas que necessitam de uma componente de detecção.

## Faster R-CNN
A imagem é usada diretamente como entrada para gerar um mapa de características convolucional. Neste modelo, porém, ao invés de realizar uma busca seletiva sobre este mapa, uma segunda rede neural, separada, é usada para predizer regiões candidatas. É chamada de Region Proposal Network (RPN). RPN usa uma mini-rede neural baseada em uma janela deslizante que analisa a imagem de entrada e é invariante a translação. Para evitar excesso de propostas, supressão de não-máximos é realizada já neste estágio. Essas regiões-candidatas, da mesma forma que no Fast-R-CNN, são reformatadas em imagens quadradas de tamanho fixo através de uma camada de pooling de região de interesse (RoI pooling). A partir do vetor de características gerado para cada RoI é realizada uma classificação com uma camada softmax, que prediz a categoria do objeto e a associa ao bounding box dado pelo quadrado de origem.
## YOLO (You only look once)
Toma-se uma imagem e divide-se-a em um grid SxS de células,usando o grid como referência, gera-se m bounding boxes,bounding boxes com probabilidade acima de um limiar são selecionados e usados para localizar o objeto dentro da imagem. Sua maior falha é inacurácia com objetos pequenos na imagem. Cada célula do grid é usada para predizer B bounding boxes (bbox) e C probabilidades de classe. Uma predição de bbox possui 5 componentes: (x, y, w, h, confiança). As coordenadas (x, y) representam o centro do bbox relativo à localização da célula (se o centro de um bbox não cair em uma célula ela não será responsável por ele e não vai representá-lo -> células só possuem uma referência a objetos cujo centro cair dentro delas). Essa coordenadas são normalizadas para [0, 1]. As dimensões do bbox (w, h) também são normalizadas para [0, 1], relativamente ao tamanho da imagem.
